In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd


def xavier_init(size):
    in_dim = size[0]
    out_dim = size[1]
    xavier_stdev = np.sqrt(2/(in_dim + out_dim))
    return tf.Variable(tf.random.truncated_normal([in_dim*out_dim], stddev=xavier_stdev), dtype = tf.float32)

def initialize_NN(layers):
    num_layers = len(layers)
    weights = tf.Variable([], dtype=tf.float32)
    biases = tf.Variable([], dtype=tf.float32)
    for l in range(len(layers)-1):
        W = xavier_init(size = [layers[l], layers[l+1]])
        b = tf.Variable(tf.zeros([layers[l+1]], dtype=tf.float32), tf.float32)
        weights = tf.concat([weights, W], 0)
        biases = tf.concat([biases, b], 0)
    return tf.concat([weights, biases], 0)

def convert_params(params):
    lower_range = 0
    upper_range = lower_range + layers[0]*layers[1]
    weights = []
    for l in range(1, len(layers)-1):
        weights.append(tf.reshape(params[lower_range:upper_range], [layers[l-1],layers[l]]))
        lower_range = upper_range
        upper_range += layers[l]*layers[l+1]
    weights.append(tf.reshape(params[lower_range:upper_range], [layers[-2], layers[-1]]))
    biases = []
    for l in range(1, len(layers)):
        lower_range = upper_range
        upper_range += layers[l]
        biases.append(tf.reshape(params[lower_range:upper_range],[1,layers[l]]))
    return weights, biases

def predict(weights, biases, inputs):
    num_layers = len(weights) + 1
    H = (2.0*(inputs-lb)/(ub-lb)-1.0)
    H = tf.cast(H, tf.float32)
    for l in range(num_layers-2):
        W = weights[l]
        b = biases[l]
        H = tf.tanh(tf.add(tf.matmul(H,W), b))
    W = weights[-1]
    b = biases[-1]
    return tf.add(tf.matmul(H, W), b)

<tf.Tensor: shape=(2181,), dtype=float32, numpy=
array([ 0.42420956, -0.21260385,  0.52530694, ...,  0.        ,
        0.        ,  0.        ], dtype=float32)>

In [4]:
import scipy.integrate as integrate

# Initialize matrices/constants
vis = .01/np.pi # Viscocity term in Burger's equation
n_x = 500 # Number of u(x, t) elements on x
X = np.zeros(n_x)
Y1 = np.zeros(n_x)
Y2 = np.zeros(n_x)
Y3 = np.zeros(n_x)
Y4 = np.zeros(n_x)
Y5 = np.zeros(n_x)
Y6 = np.zeros(n_x)
Y7 = np.zeros(n_x)
Y8 = np.zeros(n_x)
Y9 = np.zeros(n_x)
Y_ani = np.zeros(n_x)

# Analytical solution for Burger's equation with periodic boundary conditions 
# at x = [-5,5] and initial condition u(x,0) = -sin(pi*x)
def f_cole(y):
    return np.exp(-np.cos(np.pi*y)/(2*np.pi*vis))

def integrand1(eta, x, t):
    return np.sin(np.pi*(x-eta))*f_cole(x-eta)*np.exp(-eta**2/(4*vis*t))

def integrand2(eta, x, t):
    return f_cole(x-eta)*np.exp(-eta**2/(4*vis*t))

def u_anal(x, t):
    if t == 0:
        return -np.sin(np.pi*x)
    else:
        I1 = integrate.quad(integrand1, -np.inf, np.inf, args=(x,t))[0]
        I2 = integrate.quad(integrand2, -np.inf, np.inf, args=(x,t))[0]
        return -I1/I2
    
def gen_train(max_t, set_size=500):
    m = int(set_size/2)
    tr_init = np.zeros([m,3])
    tr_bound = np.zeros([m,3])
    
    tr_init[:,1] = np.linspace(-5, 5, m)
    tr_bound[:,1] = np.ones(m) - 2*np.random.randint(2, size=m)
    tr_bound[:,2] = np.linspace(0, max_t, m)
    for i in range(m):
        tr_init[i][0] = u_anal(tr_init[i][1], 0)
        tr_bound[i][0] = u_anal(tr_bound[i][1], tr_bound[i][2])
    training = np.append(tr_init, tr_bound, axis=0)
    df = pd.DataFrame(training, columns=['u(x,t)', 'x', 't'])
    return df

train_df = gen_train(500)

<ipython-input-4-3651db3b2637>:33: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  I1 = integrate.quad(integrand1, -np.inf, np.inf, args=(x,t))[0]
<ipython-input-4-3651db3b2637>:33: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  I1 = integrate.quad(integrand1, -np.inf, np.inf, args=(x,t))[0]


In [29]:
X_train = tf.cast(tf.convert_to_tensor(train_df.iloc[:,1:]), dtype=tf.float32)
Y_train = tf.cast(tf.reshape(tf.convert_to_tensor(train_df.iloc[:,0]), [500,1]), tf.float32)

ub = (X_train).numpy().max(0)
lb = (X_train).numpy().min(0)

def loss(parameters, callback=False):
    lambda_1 = parameters[0]
    parameters = parameters[1:]
    x_tf = tf.reshape(X_train[:,0], [500, 1])
    t_tf = tf.reshape(X_train[:,1], [500, 1])
    
    weights, biases = convert_params(parameters)
    with tf.GradientTape() as tape:
        tape.watch(x_tf)
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(x_tf)
            tape2.watch(t_tf)
            u_pred = predict(weights,  biases, tf.concat([x_tf, t_tf], 1))
        u_x = tape2.gradient(u_pred, x_tf)
        u_t = tape2.gradient(u_pred, t_tf)
    u_xx = tape.gradient(u_x, x_tf)
    f = u_t + u_pred*u_x - lambda_1*u_xx
    f_error = tf.reduce_sum(tf.square(f))
    pred_error = tf.reduce_sum(tf.square(u_pred - Y_train))
    if callback:
        print("F_Error: {}\t Pred_Error: {}\tLambda: {}".format(f_error, pred_error, lambda_1))
    return f_error + pred_error

In [ ]:
layers = [2, 20, 20, 20, 20, 20, 20, 1]
parameters = initialize_NN(layers)
optimizer = tf.keras.optimizers.Adam()
lambda_val = tf.convert_to_tensor([0.0], dtype=tf.float32)
parameters = tf.concat([lambda_val, parameters], 0)
parameters = tf.Variable(parameters)

adam_loss = lambda: loss(parameters)

for itr in range(250000):
    optimizer.minimize(adam_loss, parameters)
    if itr % 10 == 0:
        itr_loss = loss(parameters, callback=True)
        print("Itr: {}\tLoss: {}".format(itr, itr_loss))

F_Error: 0.0014149994822219014	 Pred_Error: 122.88314819335938	Lambda: 0.0009995113359764218
Itr: 0	Loss: 122.88455963134766
F_Error: 0.0017029718728736043	 Pred_Error: 122.80024719238281	Lambda: 0.010570354759693146
Itr: 10	Loss: 122.80194854736328
F_Error: 0.0019358128774911165	 Pred_Error: 122.22692108154297	Lambda: 0.019790204241871834
Itr: 20	Loss: 122.2288589477539
F_Error: 0.0015811561606824398	 Pred_Error: 122.15953063964844	Lambda: 0.029032262042164803
Itr: 30	Loss: 122.1611099243164
F_Error: 0.0018195579759776592	 Pred_Error: 122.13839721679688	Lambda: 0.037781305611133575
Itr: 40	Loss: 122.14021301269531
F_Error: 0.0017329980619251728	 Pred_Error: 122.0871810913086	Lambda: 0.046181242913007736
Itr: 50	Loss: 122.08891296386719
F_Error: 0.0018920076545327902	 Pred_Error: 122.06037902832031	Lambda: 0.05395260453224182
Itr: 60	Loss: 122.06227111816406
F_Error: 0.0019049957627430558	 Pred_Error: 122.0274658203125	Lambda: 0.061127450317144394
Itr: 70	Loss: 122.02937316894531
F_Err

F_Error: 0.3538365066051483	 Pred_Error: 116.64423370361328	Lambda: -0.16886261105537415
Itr: 670	Loss: 116.9980697631836
F_Error: 0.3547861576080322	 Pred_Error: 116.63428497314453	Lambda: -0.1700933277606964
Itr: 680	Loss: 116.98907470703125
F_Error: 0.35821980237960815	 Pred_Error: 116.62258911132812	Lambda: -0.17144477367401123
Itr: 690	Loss: 116.9808120727539
F_Error: 0.3608214855194092	 Pred_Error: 116.61227416992188	Lambda: -0.17279501259326935
Itr: 700	Loss: 116.97309875488281
F_Error: 0.36160415410995483	 Pred_Error: 116.60364532470703	Lambda: -0.17413602769374847
Itr: 710	Loss: 116.96524810791016
F_Error: 0.36187833547592163	 Pred_Error: 116.59600830078125	Lambda: -0.17550767958164215
Itr: 720	Loss: 116.9578857421875
F_Error: 0.40338489413261414	 Pred_Error: 118.81005859375	Lambda: -0.17685271799564362
Itr: 730	Loss: 119.21343994140625
F_Error: 0.29094794392585754	 Pred_Error: 117.14617156982422	Lambda: -0.1750783920288086
Itr: 740	Loss: 117.43711853027344
F_Error: 0.31790632

F_Error: 0.4217323362827301	 Pred_Error: 116.27276611328125	Lambda: -0.22541269659996033
Itr: 1340	Loss: 116.69449615478516
F_Error: 0.39837878942489624	 Pred_Error: 116.36011505126953	Lambda: -0.22662962973117828
Itr: 1350	Loss: 116.75849151611328
F_Error: 0.39402592182159424	 Pred_Error: 116.26933288574219	Lambda: -0.22725602984428406
Itr: 1360	Loss: 116.66336059570312
F_Error: 0.3890865445137024	 Pred_Error: 116.27175903320312	Lambda: -0.22805646061897278
Itr: 1370	Loss: 116.66084289550781
F_Error: 0.40269795060157776	 Pred_Error: 116.240966796875	Lambda: -0.2293747514486313
Itr: 1380	Loss: 116.64366149902344
F_Error: 0.401064395904541	 Pred_Error: 116.2281494140625	Lambda: -0.23102885484695435
Itr: 1390	Loss: 116.62921142578125
F_Error: 0.40474599599838257	 Pred_Error: 116.21239471435547	Lambda: -0.23247331380844116
Itr: 1400	Loss: 116.61714172363281
F_Error: 0.4083392918109894	 Pred_Error: 116.1968765258789	Lambda: -0.2338801920413971
Itr: 1410	Loss: 116.60521697998047
F_Error: 0.

F_Error: 1.108445167541504	 Pred_Error: 113.63042449951172	Lambda: -0.279371440410614
Itr: 2010	Loss: 114.7388687133789
F_Error: 1.1141225099563599	 Pred_Error: 113.5689468383789	Lambda: -0.2768869996070862
Itr: 2020	Loss: 114.68306732177734
F_Error: 1.1009448766708374	 Pred_Error: 113.56991577148438	Lambda: -0.2733699679374695
Itr: 2030	Loss: 114.67086029052734
F_Error: 1.158400297164917	 Pred_Error: 113.50159454345703	Lambda: -0.2705938518047333
Itr: 2040	Loss: 114.65999603271484
F_Error: 1.1634571552276611	 Pred_Error: 113.4893569946289	Lambda: -0.26905348896980286
Itr: 2050	Loss: 114.65281677246094
F_Error: 1.1792455911636353	 Pred_Error: 113.46929168701172	Lambda: -0.26809239387512207
Itr: 2060	Loss: 114.6485366821289
F_Error: 1.1900101900100708	 Pred_Error: 113.45520782470703	Lambda: -0.26731184124946594
Itr: 2070	Loss: 114.64521789550781
F_Error: 1.1989535093307495	 Pred_Error: 113.44356536865234	Lambda: -0.2666459083557129
Itr: 2080	Loss: 114.64251708984375
F_Error: 1.203651785

F_Error: 1.2595168352127075	 Pred_Error: 113.34386444091797	Lambda: -0.26634231209754944
Itr: 2680	Loss: 114.60337829589844
F_Error: 1.2599997520446777	 Pred_Error: 113.3429183959961	Lambda: -0.2663840055465698
Itr: 2690	Loss: 114.60292053222656
F_Error: 1.2604788541793823	 Pred_Error: 113.34200286865234	Lambda: -0.26642319560050964
Itr: 2700	Loss: 114.60247802734375
F_Error: 1.2609891891479492	 Pred_Error: 113.3410415649414	Lambda: -0.26646000146865845
Itr: 2710	Loss: 114.6020278930664
F_Error: 1.2614656686782837	 Pred_Error: 113.34011840820312	Lambda: -0.2664944529533386
Itr: 2720	Loss: 114.6015853881836
F_Error: 1.2619675397872925	 Pred_Error: 113.33916473388672	Lambda: -0.26652631163597107
Itr: 2730	Loss: 114.60113525390625
F_Error: 1.2625800371170044	 Pred_Error: 113.33811950683594	Lambda: -0.2665555775165558
Itr: 2740	Loss: 114.60070037841797
F_Error: 1.2643535137176514	 Pred_Error: 113.33790588378906	Lambda: -0.26658132672309875
Itr: 2750	Loss: 114.60225677490234
F_Error: 1.2928

F_Error: 1.3169496059417725	 Pred_Error: 113.22392272949219	Lambda: -0.25448235869407654
Itr: 3350	Loss: 114.5408706665039
F_Error: 1.318679928779602	 Pred_Error: 113.21976470947266	Lambda: -0.2540033757686615
Itr: 3360	Loss: 114.53844451904297
F_Error: 1.3205420970916748	 Pred_Error: 113.21530151367188	Lambda: -0.2534777522087097
Itr: 3370	Loss: 114.53584289550781
F_Error: 1.3225247859954834	 Pred_Error: 113.2105712890625	Lambda: -0.2529072165489197
Itr: 3380	Loss: 114.53309631347656
F_Error: 1.324597954750061	 Pred_Error: 113.20555877685547	Lambda: -0.2522898018360138
Itr: 3390	Loss: 114.53015899658203
F_Error: 1.3266856670379639	 Pred_Error: 113.20032501220703	Lambda: -0.2516193985939026
Itr: 3400	Loss: 114.52700805664062
F_Error: 1.328831672668457	 Pred_Error: 113.19483947753906	Lambda: -0.2508928179740906
Itr: 3410	Loss: 114.52367401123047
F_Error: 1.3310449123382568	 Pred_Error: 113.18903350830078	Lambda: -0.2501075565814972
Itr: 3420	Loss: 114.52008056640625
F_Error: 1.333308219

F_Error: 11.807856559753418	 Pred_Error: 41.30086135864258	Lambda: 0.015088871121406555
Itr: 4020	Loss: 53.10871887207031
F_Error: 12.005374908447266	 Pred_Error: 41.01591873168945	Lambda: 0.014474227093160152
Itr: 4030	Loss: 53.02129364013672
F_Error: 12.234522819519043	 Pred_Error: 40.360870361328125	Lambda: 0.015214172191917896
Itr: 4040	Loss: 52.595394134521484
F_Error: 12.218217849731445	 Pred_Error: 39.93550491333008	Lambda: 0.01585773006081581
Itr: 4050	Loss: 52.153724670410156
F_Error: 12.381600379943848	 Pred_Error: 39.581275939941406	Lambda: 0.016179699450731277
Itr: 4060	Loss: 51.96287536621094
F_Error: 12.717742919921875	 Pred_Error: 39.354225158691406	Lambda: 0.016529370099306107
Itr: 4070	Loss: 52.07196807861328
F_Error: 12.452786445617676	 Pred_Error: 39.8287467956543	Lambda: 0.01652565784752369
Itr: 4080	Loss: 52.281532287597656
F_Error: 12.461638450622559	 Pred_Error: 38.96928024291992	Lambda: 0.016533488407731056
Itr: 4090	Loss: 51.4309196472168
F_Error: 12.6959629058

In [ ]:
import tensorflow_probability as tfp

def val_and_gradient(params)
    print("Loss: {}".format(loss(params, callback=True)))
    return tfp.math.value_and_gradient(loss, params)

optim_results = tfp.optimizer.lbfgs_minimize(val_and_gradient, initial_position=parameters,  max_iterations=100000, max_line_search_iterations=50000)

print(optim_results.converged)
parameters = optim_results.position

In [28]:
.01/np.pi

0.003183098861837907